In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('../datasets/IRIS.csv')

df['target'] = df['species'].replace({
    'Iris-setosa': 0,
    'Iris-versicolor': 1,
    'Iris-virginica': 2
})

df.head()

,sepal_length,sepal_width,petal_length,petal_width,species,target
0,5.1,3.5,1.4,0.2,Iris-setosa,0
1,4.9,3.0,1.4,0.2,Iris-setosa,0
2,4.7,3.2,1.3,0.2,Iris-setosa,0
3,4.6,3.1,1.5,0.2,Iris-setosa,0
4,5.0,3.6,1.4,0.2,Iris-setosa,0


In [3]:
import torch

inputs = torch.tensor(
    df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].values,
    dtype=torch.float32
)

targets = torch.tensor(
    df['target'].values,
    dtype=torch.int64
)

In [4]:
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(inputs, targets)
dataloader = DataLoader(dataset, shuffle=True, batch_size=4)

In [5]:
torch.manual_seed(4050)
batch0 = next(iter(dataloader))
batch0

[tensor([[6.7000, 3.1000, 4.4000, 1.4000],
         [4.8000, 3.1000, 1.6000, 0.2000],
         [6.7000, 2.5000, 5.8000, 1.8000],
         [4.5000, 2.3000, 1.3000, 0.3000]]),
 tensor([1, 0, 2, 0])]

In [6]:
import torchmetrics

def evaluate(model):
    accuracy = torchmetrics.classification.Accuracy(task='multiclass', num_classes=3)
    x, y_exp = dataset[:]
    y = model(x)
    return accuracy(y, y_exp)

In [7]:
import lightning as L   # torch lightning
import torchinfo        # torch info
from torch import nn    # neural network layers
from torch import optim # optimizers

In [8]:
class LinearClassifier(L.LightningModule):
    def __init__(self):
        super().__init__()
        
        # define the `nn` as the neural network. In this case, it
        # should be a simple nn.Linear(...) layer.
        self.nn = nn.Linear(4, 3)
        
        # define the loss function to be used.
        # It should be nn.CrossEntropyLoss(...)
        self.loss_fn = nn.CrossEntropyLoss()
        
    def forward(self, batch):
        # return the output which should be
        # a 3D vector corresponding to the logits of the
        # three class predictions.
        return self.nn(batch)
    
    def training_step(self, batch):
        # - The batch is a tuple (x, targets), where
        #   x is the input vectors, and targets the integer labels.
        # - return the loss value for the current batch.
        # - the loss is computed using forward(...) and loss_fn(...)
        x, targets = batch
        logits = self.forward(x)
        loss = self.loss_fn(logits, targets)
        return loss
    
    def configure_optimizers(self):
        # returns an optimizer that is configured to
        # optimize `self.parameters()`.
        # Use the optim.Adam(...)
        return optim.Adam(self.parameters(), lr=0.01)

In [9]:
class MLP(LinearClassifier):
    def __init__(self, hidden:int):
        super().__init__()
        self.nn = nn.Sequential(
            # linear layer
            nn.Linear(4, hidden),
            # ReLU
            nn.ReLU(),
            # output linear layer
            nn.Linear(hidden, 3)
        )

In [10]:
torchinfo.summary(MLP(10))

Layer (type:depth-idx)                   Param #
MLP                                      --
├─Sequential: 1-1                        --
│    └─Linear: 2-1                       50
│    └─ReLU: 2-2                         --
│    └─Linear: 2-3                       33
├─CrossEntropyLoss: 1-2                  --
Total params: 83
Trainable params: 83
Non-trainable params: 0

In [11]:
torch.manual_seed(0)
model = MLP(10)
trainer = L.Trainer(max_epochs=10)

trainer.fit(model, train_dataloaders=dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name    | Type             | Params | Mode 
-----------------------------------------------------
0 | nn      | Sequential       | 83     | train
1 | loss_fn | CrossEntropyLoss | 0      | train
-----------------------------------------------------
83        Trainable params
0         Non-trainable params
83        Total params
0.000     Total estimated model params size (MB)
5         Modules in train mode
0         Modules in eval mode


Epoch 9: 100%|██████████| 38/38 [00:00<00:00, 413.05it/s, v_num=2]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 38/38 [00:00<00:00, 387.75it/s, v_num=2]


In [12]:
evaluate(model)

tensor(0.9533)

In [13]:
torch.save(model.state_dict(), "my_model.pth")
model.eval()

MLP(
  (nn): Sequential(
    (0): Linear(in_features=4, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=3, bias=True)
  )
  (loss_fn): CrossEntropyLoss()
)

In [14]:
import onnx

torch.onnx.export(
    model,
    torch.randn(1, 4),
    "my_model.onnx",
    export_params=True,
    opset_version=11,
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}}
)